In [12]:
import gdal
import osgeo as osr
import geopandas as pd
import numpy as np
import cv2
import random

In [26]:
def random_colour():
    colour=[]
    for _ in range(3):
        colour.append(random.randint(1, 255))
    return tuple(colour)


def geo2imagexy(dataset, x, y):
    
    trans = dataset.GetGeoTransform()
    a = np.array([[trans[1], trans[2]], [trans[4], trans[5]]])
    b = np.array([x - trans[0], y - trans[3]])
    return np.linalg.solve(a, b)

def getSRSPair(dataset):
    prosrs = osr.SpatialReference()
    prosrs.ImportFromWkt(dataset.GetProjection())
    geosrs = prosrs.CloneGeogCS()
    return prosrs, geosrs
def lonlat2geo(dataset, lon, lat):
   
    prosrs, geosrs = getSRSPair(dataset)
    ct = osr.CoordinateTransformation(geosrs, prosrs)
    coords = ct.TransformPoint(lon, lat)
    return coords[:2]
def run_png(tif_path,shp_path,out_path):
    shape_data = pd.GeoDataFrame.from_file(shp_path)
    tif_data = gdal.Open(tif_path)
    tif_datashape = tif_data.ReadAsArray()
    print(tif_datashape.shape)
    level_xiao=[]
    shapes=[]
    kkt = []
    for b in shape_data['geometry']:
        if b.geom_type == "MultiPolygon":
            for pg in b:
                for j in pg.exterior.coords[:]:
                    #print("#"*20)
                    print(j)
                    (x,y,_)=j
                    #(xt,yt)=lonlat2geo(tif_data,x,y)
                    (xt,yt)=geo2imagexy(tif_data,x,y)
                    level_xiao.append((xt,yt))
                shapes.append(level_xiao)
                level_xiao=[]
        else:
            for ji in b.exterior.coords[:]:
                (x,y)=ji[0],ji[1]
                #(xt,yt)=lonlat2geo(tif_data,x,y)
                (xt,yt)=geo2imagexy(tif_data,x,y)
                level_xiao.append((xt,yt))
            shapes.append(level_xiao)
            level_xiao=[]
    images = np.zeros((tif_datashape.shape[1],tif_datashape.shape[2],3))
    colours = set()
    for polygon in shapes:
        np_polygon=np.array([polygon], dtype=np.int32)
        colour=random_colour()
        
        while colour in colours:
            colour=random_colour()
        
        colours.add(colour)
        cv2.fillPoly(images,np_polygon,colour)
    cv2.imwrite(out_path,images)
    print(np.unique(images,return_counts=True))

In [27]:
random_colour()

(104, 232, 244)

In [28]:
run_png("./aerial_sampled.tif", 
        "./shapefiles/derelict_sampled.shp", 
        "derelict_mask.png") 

(3, 9252, 14056)
(3, 9252, 14056)
(array([  0.,   2.,   5.,   6.,   7.,  10.,  12.,  15.,  17.,  19.,  24.,
        25.,  26.,  27.,  28.,  31.,  32.,  38.,  40.,  47.,  48.,  50.,
        51.,  52.,  53.,  57.,  58.,  59.,  61.,  63.,  64.,  66.,  68.,
        73.,  74.,  75.,  77.,  78.,  79.,  80.,  81.,  83.,  87.,  89.,
        92.,  94.,  95.,  96., 100., 101., 103., 105., 106., 107., 114.,
       115., 116., 117., 119., 121., 124., 125., 127., 129., 131., 132.,
       133., 134., 136., 138., 139., 143., 144., 147., 154., 156., 159.,
       166., 169., 170., 176., 177., 178., 179., 181., 194., 197., 198.,
       200., 201., 205., 207., 208., 209., 215., 216., 218., 220., 225.,
       227., 228., 235., 237., 238., 239., 240., 241., 244., 246., 247.,
       248., 251., 252., 253., 254., 255.]), array([340651905,    358424,   3184573,     92446,     66651,    269244,
          471040,    248859,    135061,     30575,    102163,   1158256,
         1560094,    131571,   2758015,     

In [29]:
run_png("./aerial_sampled.tif",
        "./shapefiles/vacant_sampled.shp",
        "vacant_mask.png")

(3, 9252, 14056)
(3, 9252, 14056)
(array([  0.,   4.,   6.,   7.,  13.,  14.,  20.,  24.,  26.,  27.,  28.,
        29.,  30.,  34.,  35.,  37.,  38.,  42.,  45.,  46.,  47.,  50.,
        51.,  52.,  57.,  58.,  61.,  63.,  64.,  66.,  67.,  71.,  73.,
        75.,  76.,  77.,  79.,  82.,  84.,  88.,  90.,  95.,  97.,  99.,
       104., 105., 109., 110., 117., 118., 121., 123., 124., 125., 128.,
       130., 132., 134., 137., 140., 143., 152., 156., 157., 160., 163.,
       166., 168., 173., 175., 181., 182., 183., 184., 187., 188., 189.,
       190., 192., 193., 196., 197., 198., 202., 205., 208., 209., 212.,
       214., 217., 218., 225., 231., 233., 235., 236., 239., 241., 243.,
       244., 248., 253.]), array([377996934,     22984,     42015,    116990,     72715,     25745,
          116990,    105236,     66783,    232006,    114730,     26390,
          221062,    118385,     83673,     82333,    161567,     49436,
          177729,    218387,     61666,     83673,     79139, 

In [30]:
imgs = cv2.imread("vacant_mask.png",0)
imgm = cv2.imread("derelict_mask.png",0)
tmp = np.zeros(imgs.shape)
tmp[imgs==255]=1 # label mask 1 to 1
tmp[imgm==255]=2 # label mask 2 to 2
#cv2.imwrite("vacant_1_derelict_2.png",tmp)